# Importing the relevent files

In [27]:
import keras
import tensorflow as tf
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from os import listdir
from PIL import Image
import glob
import pickle
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential,load_model
from keras.layers import GRU, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from pickle import dump
#from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
#from keras.applications.vgg16 import preprocess_input
from keras.models import Model

In [41]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
import cv2

In [2]:
"""from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
#from tensorflow import ConfigProto
#from tensorflow import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)"""

# creating functions for loading files and creating features

In [3]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [4]:
def extract_features(directory):
	# load the model
	model = InceptionV3()
	# re-structure the model
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# summarize
	print(model.summary())
	# extract features from each photo
	features = dict()
	for name in listdir(directory):
		# load an image from file
		filename = directory + '\\' + name
		image = load_img(filename, target_size=(299, 299))
		# convert the image pixels to a numpy array
		image = img_to_array(image)
		# reshape data for the model
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		# prepare the image for the Inception model
		image = preprocess_input(image)
		# get features
		feature = model.predict(image, verbose=0)
		# get image id
		image_id = name.split('.')[0]
		# store feature
		features[image_id] = feature
		print('>%s' % name)
	return features

In [5]:
"""directory = 'C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\Flicker8k_Dataset'
feature = extract_features(directory)
print('Extracted Features: %d' % len(feature))
# save to file
dump(feature, open('C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\features.pkl', 'wb'))"""

"directory = 'C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\Flicker8k_Dataset'\nfeature = extract_features(directory)\nprint('Extracted Features: %d' % len(feature))\n# save to file\ndump(feature, open('C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\features.pkl', 'wb'))"

# loading ,cleaning the description and creating vocab

In [6]:
######################## loading file code ####################################################
def load_file(doc):
    file = open(doc,'r')
    text = file.read()
    file.close()
    return text
######################## loading description part ##############################################
def load_describe(doc):
    mapping = dict()
    for l in doc.split('\n'):
        tk = l.split(' ')
        if(len(l)<2):
            continue
        img_id,img_des = tk[0],tk[1:]
        img_id = img_id.split('.')[0]
        img_des = ' '.join(img_des)
        
        if(img_id not in mapping):
            mapping[img_id]=list()
        mapping[img_id].append(img_des)
    return mapping

########################## cleaning the description part ############################################
def cleaning_descibe_data(describe):
    trans = str.maketrans('','',string.punctuation)
    for key,data in describe.items():
        for k in range(len(data)):
            store = data[k]
           
            store = store.split()                         # tokenize
            
            store = [word.lower() for word in store]      # lowering
            
            store = [word.translate(trans) for word in store] # remove punchuation
            
            store = [word for word in store if len(word)>1]
            
            store = [word for word in store if word.isalpha()]
            
            data[k] = ' '.join(store)
            
######################## creating vocab ######################

def vocab_part(describe):
    desc=set()
    for key in describe.keys():
        [desc.update(des.split()) for des in describe[key]]
    return desc

##################### saving the description #############
def store_describe(describe,f_name):
    line=list()
    for key,data in describe.items():
        for d in data:
            line.append(key+ ' ' +d)
    data='\n'.join(line)
    file=open(f_name,'w')
    file.write(data)
    file.close()

In [7]:
describe = load_file('C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\Flickr8k.token.txt') 
descript = load_describe(describe)
cleaning_descibe_data(descript)
vocab = vocab_part(descript)
#store_describe(descript,'C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\describe.txt')

# checking the vocabulary length from description

In [8]:
print('len describe %d' % len(descript))
print('len vocab %d' % len(vocab))

len describe 8092
len vocab 8680


In [9]:
descript['1000268201_693b08cb0e']

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin']

# Creating funtions for loading the description as per the images

In [10]:
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

In [11]:
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions



def load_photo_features(filename, dataset):
	# load all features
	all_features = pickle.load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

In [12]:
file = 'C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\Flickr_8k.trainImages.txt'
train = load_set(file)
print('Dataset: %d' % len(train))
train_des = load_clean_descriptions('C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\describe.txt',train)
print('train_des: %d' % len(train_des))

train_feature = load_photo_features('C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\features.pkl',train)
print('Photos: train=%d' % len(train_feature))

Dataset: 6000
train_des: 6000
Photos: train=6000


In [14]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 

In [36]:
t_tokenize = create_tokenizer(train_des)

In [37]:
pickle.dump(t_tokenize,open('C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\t_tokenizer.pkl','wb'))

In [32]:
tokenizer = create_tokenizer(train_des)
vocab_size = len(tokenizer.word_index)+1
print('vocab_size: %d'%vocab_size)

vocab_size: 7507


# creating sequences for text generation

In [16]:
# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each description for the image
	for desc in desc_list:
		# encode the sequence
		seq = tokenizer.texts_to_sequences([desc])[0]
		# split one sequence into multiple X,y pairs
		for i in range(1, len(seq)):
			# split into input and output pair
			in_seq, out_seq = seq[:i], seq[i]
			# pad input sequence
			in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
			# encode output sequence
			out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
			# store
			X1.append(photo)
			X2.append(in_seq)
			y.append(out_seq)
	return array(X1), array(X2), array(y)

In [30]:
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# Checking the max length for which we can generate the caption

In [31]:
max_length = max_length(train_des)
print(max_length)

33


# Now defining the model for image captioning

In [19]:
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(2048,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = GRU(256)(se2)
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# summarize model
	print(model.summary())
	#keras.utils.plot_model(model, to_file='model.png', show_shapes=True)
	return model

In [20]:
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
	# loop for ever over images
	while 1:
		for key, desc_list in descriptions.items():
			# retrieve the photo feature
			photo = photos[key][0]
			in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
			yield [in_img, in_seq], out_word

In [19]:
x1_train,x2_train,ytrain = create_sequences(tokenizer , max_length, train_des, train_feature, vocab_size)

In [29]:
file = 'C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\Flickr_8k.testImages.txt'
test = load_set(file)
print('Dataset: %d' % len(test))
test_des = load_clean_descriptions('C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\describe.txt',test)
print('test_des: %d' % len(test_des))

test_feature = load_photo_features('C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\features.pkl',test)
print('Photos: train=%d' % len(test_feature))

Dataset: 1000
test_des: 1000
Photos: train=1000


In [21]:
x1_test,x2_test,ytest = create_sequences(tokenizer,max_length,test_des,test_feature,vocab_size)

# final structure of model below

In [22]:
model = define_model(vocab_size,max_length)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 33)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 33, 256)      1921792     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
____________________________________________________________________________________________

In [36]:
# test the data generator
generator = data_generator(train_des, train_feature, tokenizer, max_length, vocab_size)
inputs, outputs = next(generator)
print(inputs[0].shape)
print(inputs[1].shape)
print(outputs.shape)

(47, 2048)
(47, 33)
(47, 7507)


# training the model and saving the model

In [ ]:
epochs = 20
steps = len(train_des)
for i in range(epochs):
	# create the data generator
	generator = data_generator(train_des, train_feature, tokenizer, max_length, vocab_size)
	# fit for one epoch
	model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
	# save model
	model.save('C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\model_weights\\model_' + str(i) + '.h5')

Epoch 1/1
6000/6000 [==============================] - 2322s 387ms/step - loss: 4.5220
Epoch 1/1
6000/6000 [==============================] - 1722s 287ms/step - loss: 3.7394
Epoch 1/1
6000/6000 [==============================] - 2141s 357ms/step - loss: 3.4785
Epoch 1/1
6000/6000 [==============================] - 2591s 432ms/step - loss: 3.3290
Epoch 1/1
6000/6000 [==============================] - 2583s 430ms/step - loss: 3.2310
Epoch 1/1
6000/6000 [==============================] - 1714s 286ms/step - loss: 3.1582
Epoch 1/1
6000/6000 [==============================] - 1723s 287ms/step - loss: 3.1087
Epoch 1/1
6000/6000 [==============================] - 2778s 463ms/step - loss: 3.0715
Epoch 1/1
6000/6000 [==============================] - 2271s 378ms/step - loss: 3.0426
Epoch 1/1
6000/6000 [==============================] - 2143s 357ms/step - loss: 3.0198
Epoch 1/1
 444/6000 [=>............................] - ETA: 26:27 - loss: 2.9996

In [21]:
model_path='C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\model_weights\\model_9.h5'

In [22]:
def word_id(num,tokenizer):
    for word,idx in tokenizer.word_index.items():
        if(idx == num):
            return word
    return None

def generate_describe(model,tokenizer,photo,max_length):
    txt = 'startseq'
    for i in range(max_length):
        seq = tokenizer.texts_to_sequences([txt])[0]
        seq = pad_sequences([seq],maxlen=max_length)
        pred= model.predict([photo,seq],verbose=0)
        pred=np.argmax(pred)
        
        word = word_id(pred,tokenizer)
        
        if(word is None):
            break
        txt += ' '+ word
        if(word == 'endseq'):
            break
    return txt

In [34]:
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_describe(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [28]:
f_model = load_model(model_path)

C:\Users\gaurav\miniconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [35]:
evaluate_model(f_model,test_des,test_feature,tokenizer,max_length)

BLEU-1: 0.513198
BLEU-2: 0.279901
BLEU-3: 0.187234
BLEU-4: 0.080185


In [38]:
tokenizer_t = pickle.load(open('C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\t_tokenizer.pkl','rb'))

In [39]:
def extract_features(filename):
	# load the model
	model = InceptionV3()
	# re-structure the model
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# load the photo
	image = load_img(filename, target_size=(299, 299))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the Inception model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature

In [64]:
file='C:\\Users\\gaurav\\miniconda3\\envs\\Datasets\\Flickr8k\\Flicker8k_Dataset\\232874193_c691df882d.jpg'
photo = extract_features(file)

In [75]:
img = cv2.imread(file)
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [71]:
describe = generate_describe(f_model,tokenizer_t,photo,max_length)

In [72]:
print(describe)

startseq dog is running through the grass endseq


In [76]:
########### done with this project #################